In [45]:
import pandas as pd

predict_df=pd.read_excel('prediction_data.xlsx')



In [47]:
predict_df

,ID,Name,Role,Transcript,Resume,Reason for decision,Job Description
0,ericri886,Eric Rich,Game Developer,Here's the simulated professional interview:\n...,Here's a sample resume for Eric Rich:\n\nEric ...,Strong technical skills in AI and ML.,We are looking for an experienced Game Develop...
1,mattre109,Matthew Reyes,IT Support Specialist,Here's the simulated interview:\n\nInterviewer...,Here's a professional resume for Matthew Reyes...,Lacks hands-on experience with cloud platforms.,We are looking for an experienced IT Support S...
2,genere966,Gene Reed,Robotics Engineer,Interview Setting: A conference room in a robo...,Here's a professional resume for Gene Reed:\n\...,Insufficient system design expertise for senio...,We're seeking a talented Robotics Engineer to ...
3,waynco331,Wayne Cohen,Blockchain Developer,Here's a simulated professional interview for ...,Here's a professional resume for Wayne Cohen:\...,Lacked leadership skills for a senior position.,Be part of a passionate team at the forefront ...
4,alexgr300,Alexander Green,Human Resources Specialist,"Interviewer: Good morning, Alexander. Thank yo...","Here's a sample resume for Alexander Green, a ...",Impressive leadership and communication abilit...,Help us build the next-generation products as ...
...,...,...,...,...,...,...,...
95,marcmc483,Marcus Mcintosh,AR/VR Developer,Here's a simulated professional interview for ...,Here's a sample resume for Marcus Mcintosh:\n\...,Lacked leadership skills for a senior position.,We're hiring a AR/VR Developer to develop and ...
96,ambewa488,Amber Walker,IT Support Specialist,Here's the simulated interview:\n\nInterviewer...,Here's a professional resume for Amber Walker:...,Solid experience in machine learning and AI.,Help us build the next-generation products as ...
97,marksh769,Mark Sherman,Business Analyst,Here's the simulated interview:\n\nInterviewer...,Here's a sample resume for Mark Sherman:\n\nMa...,Impressive leadership and communication abilit...,We are looking for an experienced Business Ana...
98,donaro891,Donald Rodriguez,IT Support Specialist,"Interviewer: Good morning, Donald. Thank you f...",Donald Rodriguez\nContact Information:\n\n* Ad...,Lacks hands-on experience with cloud platforms.,We're hiring a IT Support Specialist to develo...


In [5]:
# Calculate resume and job description similarity (Cosine Similarity)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer()
resume_jd_similarity = []
for i in range(len(predict_df)):
    resume = predict_df['Resume'][i]
    jd = predict_df['Job Description'][i]
    similarity = cosine_similarity(vectorizer.fit_transform([resume, jd]))[0, 1]
    resume_jd_similarity.append(similarity)
predict_df['resume_jd_similarity'] = resume_jd_similarity

In [6]:
# Calculate resume and transcript similarity (Cosine Similarity)
resume_transcript_similarity = []
for i in range(len(predict_df)):
    resume = predict_df['Resume'][i]
    transcript = predict_df['Transcript'][i]
    similarity = cosine_similarity(vectorizer.fit_transform([resume, transcript]))[0, 1]
    resume_transcript_similarity.append(similarity)
predict_df['resume_transcript_similarity'] = resume_transcript_similarity

In [7]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Perform sentiment analysis on each transcript
predict_df['sentiment'] = predict_df['Transcript'].apply(lambda transcript: sia.polarity_scores(transcript)['compound'])

# Overall average sentiment score
average_sentiment = predict_df['sentiment'].mean()




In [8]:
# Function to calculate lexical diversity
def lexical_diversity(text):
    words = text.split()
    return len(set(words)) / len(words)

# Compute lexical diversity for each transcript
predict_df['lexical_diversity'] = predict_df['Transcript'].apply(lexical_diversity)


In [9]:
# Length of transcript (number of words)
predict_df['transcript_length_words'] = predict_df['Transcript'].apply(lambda x: len(x.split()))


In [10]:
#Reason for Decision Length
predict_df['reason_length'] = predict_df['Reason for decision'].str.split().apply(len)


In [11]:
# Resume length (number of words)
predict_df['resume_length'] = predict_df['Resume'].apply(lambda x: len(x.split()))

In [12]:
#Word Count Ratio
predict_df['word_count_ratio'] = predict_df['transcript_length_words'] / predict_df['resume_length']


In [13]:
#Role to Transcript Similarity

def text_similarity(text1, text2):
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity

    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([text1, text2])
    return cosine_similarity(vectors[0], vectors[1])[0, 0]

predict_df['role_transcript_similarity'] = predict_df.apply(
    lambda row: text_similarity(row['Role'], row['Transcript']), axis=1
)


In [14]:
from textblob import TextBlob
predict_df['cultural_fit_sentiment'] = predict_df['Reason for decision'].apply(lambda x: TextBlob(x).sentiment.polarity)


In [15]:
#Job Description to Transcript Sentiment Gap
predict_df['jd_transcript_sentiment_gap'] = predict_df['sentiment'] - predict_df['cultural_fit_sentiment']


In [16]:
#Job Description Length
predict_df['job_desc_length'] = predict_df['Job Description'].str.split().apply(len)


In [17]:
#Role to Resume Similarity
predict_df['role_resume_similarity'] = predict_df.apply(
    lambda row: text_similarity(row['Role'], row['Resume']), axis=1
)


In [18]:
#Combined Text Similarity
predict_df['combined_text_similarity'] = (
    predict_df['resume_jd_similarity'] + predict_df['resume_transcript_similarity']
) / 2


In [19]:
#Sentiment to Lexical Diversity Ratio
predict_df['sentiment_to_diversity_ratio'] = predict_df['sentiment'] / predict_df['lexical_diversity']


In [20]:
#clarity score
import textstat
predict_df['clarity_score'] = predict_df['Transcript'].apply(lambda x: textstat.flesch_reading_ease(x))


In [21]:
#confidence score
predict_df['confidence_score'] = predict_df['Transcript'].apply(lambda x: x.count('I think') + x.count('Maybe'))


In [22]:
#Clarity and Confidence Interaction
predict_df['clarity_confidence_interaction'] = predict_df['clarity_score'] * predict_df['confidence_score']


In [23]:
#Soft Skills
from textblob import TextBlob

predict_df['soft_skills_sentiment'] = predict_df['Transcript'].apply(lambda x: TextBlob(x).sentiment.polarity)


In [24]:
# Length of transcript (number of characters)
predict_df['transcript_length_characters'] = predict_df['Transcript'].apply(len)


In [25]:
# Function to compute similarity score between Resume and Job Description
def compute_similarity(text1, text2):
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform([text1, text2])
    return cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]

# Calculate technical skill matching score
predict_df['technical_skill_match'] = predict_df.apply(lambda row: compute_similarity(row['Resume'], row['Job Description']), axis=1)


In [26]:
# Function to compute similarity score between Resume and Job Description
def compute_similarity(text1, text2):
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform([text1, text2])
    return cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]

# Calculate technical skill matching score
predict_df['technical_skill_match2'] = predict_df.apply(lambda row: compute_similarity(row['Resume'], row['Transcript']), axis=1)


In [27]:
# Function to compute similarity score between Resume and Job Description
def compute_similarity(text1, text2):
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform([text1, text2])
    return cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]

# Calculate technical skill matching score
predict_df['technical_skill_match3'] = predict_df.apply(lambda row: compute_similarity(row['Job Description'], row['Transcript']), axis=1)


In [28]:
# Job Description Experience Match (Simple matching based on keywords, could be improved)
predict_df['job_description_experience_match'] = predict_df.apply(lambda row: len(set(row['Resume'].split()) & set(row['Job Description'].split())), axis=1)


In [29]:
#job score
def job_fit_analysis(job_desc, transcript):
    # You can use similarity or keyword matching here
    job_keywords = job_desc.split()
    transcript_keywords = transcript.split()
    common_keywords = set(job_keywords).intersection(transcript_keywords)
    return len(common_keywords) / len(job_keywords)

predict_df['job_fit_score'] = predict_df.apply(lambda row: job_fit_analysis(row['Job Description'], row['Transcript']), axis=1)


In [30]:
#job description complexity
predict_df['job_desc_complexity'] = predict_df['Job Description'].apply(lambda x: textstat.flesch_reading_ease(x))

In [31]:
predict_df['num_words_in_transcript'] = predict_df['Transcript'].apply(lambda x: len(x.split()))

In [32]:
#interaction quality check
predict_df['interaction_quality'] = predict_df['num_words_in_transcript'] * predict_df['sentiment']

In [33]:
# Text complexity (resume and transcript - using a simple metric like Flesch Reading Ease)
def text_complexity(text):
    # Implement text complexity (e.g., Flesch Reading Ease)
    # Here's a placeholder function:
    return len(text.split()) / len(set(text.split()))  # A basic metric

predict_df['text_complexity_transcript'] = predict_df['Transcript'].apply(text_complexity)
predict_df['text_complexity_resume'] = predict_df['Resume'].apply(text_complexity)


In [34]:
# Create a fresh TF-IDF vectorizer and fit it to the data
loaded_vectorizer = TfidfVectorizer()
loaded_vectorizer.fit(predict_df['Job Description'].fillna(''))  # Fit on job descriptions

TfidfVectorizer()

In [35]:
# Transform text data
new_job_desc_vectors = loaded_vectorizer.transform(predict_df['Job Description'].fillna(''))
new_resume_vectors = loaded_vectorizer.transform(predict_df['Resume'].fillna(''))
new_transcript_vectors = loaded_vectorizer.transform(predict_df['Transcript'].fillna(''))


In [36]:
# Cosine similarities
predict_df['resume_jod_similarity'] = [
    cosine_similarity(new_resume_vectors[i], new_job_desc_vectors[i])[0][0] for i in range(len(predict_df))
]
predict_df['transcript_jod_similarity'] = [
    cosine_similarity(new_transcript_vectors[i], new_job_desc_vectors[i])[0][0] for i in range(len(predict_df))
]


In [38]:
import pickle

# Load the best model from the saved file
with open('xgboost_best_model.pkl', 'rb') as model_file:
    xgb_best = pickle.load(model_file)

print("XGBoost model loaded successfully.")


XGBoost model loaded successfully.


In [39]:
# List of features that match the model training features
features = [
    'num_words_in_transcript', 'resume_jd_similarity',
    'resume_transcript_similarity', 'sentiment', 'lexical_diversity',
    'transcript_length_words', 'reason_length', 'resume_length',
    'word_count_ratio', 'role_transcript_similarity',
    'cultural_fit_sentiment', 'jd_transcript_sentiment_gap',
    'job_desc_length', 'role_resume_similarity', 'combined_text_similarity',
    'sentiment_to_diversity_ratio', 'clarity_score', 'confidence_score',
    'clarity_confidence_interaction', 'soft_skills_sentiment',
    'transcript_length_characters', 'technical_skill_match',
    'technical_skill_match2', 'technical_skill_match3',
    'job_description_experience_match', 'job_fit_score',
    'job_desc_complexity', 'interaction_quality',
    'text_complexity_transcript', 'text_complexity_resume'
]

# Ensure you're selecting only the relevant columns
X = predict_df[features]

# Convert data to numeric if needed
X = X.apply(pd.to_numeric, errors='coerce')

# Make predictions using the best XGBoost model
predict_df['Selection_Status'] = xgb_best.predict(X)

# Map the result to 'Selected' or 'Not Selected'
predict_df['Selection_Status'] = predict_df['Selection_Status'].apply(lambda x: 'select' if x == 1 else 'reject')

In [40]:
# Save only specific columns (e.g., 'Selection_Status' along with other relevant ones)
predict_df[['Name','decision','Selection_Status']].to_excel('predicted_selection_status.xlsx', index=False)
print("Selected columns have been saved to 'predicted_selection_status.xlsx'.")


Selected columns have been saved to 'predicted_selection_status.xlsx'.


In [49]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

# Email credentials and setup
sender_email = "yallapuyaswanthkumar66@gmail.com"  # Your email address
receiver_email = "tharaknanda7@gmail.com"  # Receiver's email address
subject = "Predicted Selection Status"
body = "Please find attached the predicted selection status Excel file."

# Create the MIME message
msg = MIMEMultipart()
msg['From'] = sender_email
msg['To'] = receiver_email
msg['Subject'] = subject

# Attach the email body text
msg.attach(MIMEText(body, 'plain'))

# Attach the Excel file
file_path = 'predicted_selection_status.xlsx'
with open(file_path, 'rb') as file:
    part = MIMEBase('application', 'octet-stream')
    part.set_payload(file.read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', f'attachment; filename={file_path}')
    msg.attach(part)

# Connect to the SMTP server (Gmail example)
try:
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()  # Start TLS for security
        server.login(sender_email, "mvao ctgh ikue awdw")  # Login to your email account
        server.sendmail(sender_email, receiver_email, msg.as_string())  # Send email
        print("Email with attachment has been sent successfully.")
except Exception as e:
    print(f"Error: {e}")


Email with attachment has been sent successfully.
